In [1]:
import pandas as pd
from datetime import datetime
import sys
from IPython.display import clear_output

/Users/valentin/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1035: UserWarning: Duplicate key in file "/Users/valentin/anaconda/lib/python2.7/site-packages/matplotlib/mpl-data/matplotlibrc", line #513
  (fname, cnt))


In [2]:
date = []
chain_store = []
store_no = []
store_name = []
postal_no = []
EAN = []
Produkttekst = []
oms = []
volume = []
avance = []
errors = []
with open('RU udtrëk af salg af õl for 2015  - 20161026.csv') as f:
    lines = f.readlines()
    print(str(len(lines)) + " lines")
    k = len(lines)/100
    j = 0
    for l in lines:
        #clear_output()
        #print j
        #print len(splits)
        l = l.replace('4,5%' , '4.5%')
        l = l.replace('FAKTA,JENS BAGGESENSVEJ' , 'FAKTA - JENS BAGGESENSVEJ')
        splits = l.split(',')
        if len(splits) == 10:
            i = 0
        else:
            i = 1
        date.append(  datetime.strptime(splits[0].replace('\xef\xbb\xbf',''), '%Y-%m-%d') )
        chain_store.append(splits[1])
        store_no.append(int(splits[2]))
        store_name.append(splits[3])
        postal_no.append(int(splits[4+i]))
        EAN.append(splits[5+i])
        Produkttekst.append(splits[6+i])
        oms.append(float(splits[7+i]))
        volume.append(int(float(splits[8+i])))
        avance.append(int(float(splits[9+i][:-4])))

        j += 1
        if j % k == 0:
            clear_output()
            print str(int(float(j)/float(len(lines))*100.0)) + '%'

df = pd.DataFrame( {'date': date , 'chain_store' : chain_store, 'store_no': store_no, 'store_name':store_name,'postal_no':postal_no,
                   'EAN':EAN,'Produkttekst': Produkttekst, 'oms':oms, 'volume':volume, 'avance':avance})
df[:10]
print(len(df))

99%
6094460


## add weather data 

In [4]:
def zipToRegion(zip):
    
    # 1-2 Copenhagen
    if (zip >= 1000 and zip <= 2999):
        return 618800
        
    # 3 Sealand
    if (zip >= 3000 and zip <= 3699) or (zip >= 4000 and zip <= 4999):
        return 617400
        
    # 5 Fyn and the islands
    if (zip >= 5000 and zip <= 5999):
        return 612600
        
    # 6 Southern/Western Jutland
    if (zip >= 6000 and zip <= 6999):
        return 6116
    
    # skip th
    if (zip >= 7000 and zip <= 7999):
        return 0
        
    # 7 Eastern Jutland
    if (zip >= 8000 and zip <= 8999):
        return 607200
    
    # 9 Northern Jutland
    if (zip >= 9000 and zip <= 9999):
        return 603100

In [6]:
df['region'] = df['postal_no'].map(zipToRegion)
df

,EAN,Produkttekst,avance,chain_store,date,oms,postal_no,store_name,store_no,volume,region
0,5741000116975,Royal B X-mas 6x33ds,1,DagliBrugsen,2015-01-01,31.00,2300,COOP DAGLIBR BRYDES ALLE,1043,1,618800
1,5741000131084,Royal Export 6x33 ds,-13,DagliBrugsen,2015-01-01,100.00,2300,COOP DAGLIBR BRYDES ALLE,1043,4,618800
2,5741000140284,ROYAL PLATINUM 6-PAK,8,DagliBrugsen,2015-01-01,39.95,2300,COOP DAGLIBR BRYDES ALLE,1043,1,618800
3,5741000131084,Royal Export 6x33 ds,17,DagliBrugsen,2015-01-01,79.90,2400,COOP DAGLIBR BISPEPARKEN,1053,2,618800
4,5741000132463,Royal Exp. kuffert24,-19,DagliBrugsen,2015-01-01,299.85,2650,COOP LOKALBR FJELDSTEDVEJ,1082,3,618800
5,5741000131084,Royal Export 6x33 ds,9,DagliBrugsen,2015-01-01,39.95,4000,COOP DAGLIBR GEVNINGE,1099,1,617400
6,5741000116494,Royal Class. 6x33 ds,8,DagliBrugsen,2015-01-01,38.95,4000,DAGLIBRUGSEN GUNDSØMAGLE,1104,1,617400
7,5741000125021,Royal Stout 33 cl ds,6,DagliBrugsen,2015-01-01,21.50,4000,DAGLIBRUGSEN GUNDSØMAGLE,1104,2,617400
8,5741000132456,Royal Class. 24ds,-4,DagliBrugsen,2015-01-01,99.95,4000,DAGLIBRUGSEN GUNDSØMAGLE,1104,1,617400
9,5741000132463,Royal Exp. kuffert24,-6,DagliBrugsen,2015-01-01,99.95,4000,DAGLIBRUGSEN GUNDSØMAGLE,1104,1,617400


## load weather data

In [ ]:
df.to_csv('')

In [ ]:
df.to_csv('cleaned_sales+weather_data.csv')